<a href="https://colab.research.google.com/github/ChayanitBhateja/mySparkLearning/blob/master/sparkLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=50a4a785ae49808a1e369fb128778a6428049ebeb8b231ce680fe30ecf1c9049
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('linReg').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [8]:
data = spark.read.csv('Ecommerce_Customers.csv', header=True, inferSchema=True)

In [9]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
data.columns[0:-1]

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership']

In [20]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [22]:
output = assembler.transform(data)
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [26]:
finalData = output.select(['features','Yearly Amount Spent'])

In [27]:
train_data, test_data = finalData.randomSplit([0.7,0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean|  501.4894069966803|
| stddev|  80.27290379401934|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean| 494.56148107919705|
| stddev|  77.21923513376383|
|    min|   266.086340948469|
|    max|  689.2356997616951|
+-------+-------------------+



In [30]:
linearModel = LinearRegression(labelCol='Yearly Amount Spent')

In [32]:
linearModel = linearModel.fit(train_data)

In [38]:
# This will show the analysis results of what residuals and what error like stats.
evaluation_results = linearModel.evaluate(test_data)

In [39]:
evaluation_results.residuals.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
|-13.633881742530434|
|-1.0914947729382334|
|-18.784815222781504|
| -3.980295863717572|
|  5.541433114554366|
|-14.032282051869458|
|  2.571993310215362|
| -5.614558586837404|
|  4.246662568519071|
| -14.27054264914841|
| 17.439487262032117|
|   -2.2448561294201|
| -3.150885472375137|
| -6.460257782081669|
| -4.395580599630023|
|-11.991045112047914|
| -17.18777152326544|
|  6.277104499152244|
|-10.479046760845563|
|-17.555295915525278|
+-------------------+
only showing top 20 rows



In [40]:
evaluation_results.rootMeanSquaredError

9.821232306169378

In [41]:
evaluation_results.r2

0.9837199389038541

In [42]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.3662121671876...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8648325480987...|
|[31.9048571310136...|
+--------------------+
only showing top 20 rows



In [43]:
predictions = linearModel.transform(unlabeled_data)

In [44]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|333.56275154572404|
|[30.5743636841713...| 443.1559085310039|
|[30.8162006488763...| 284.8711561712505|
|[30.8794843441274...|494.18689584857225|
|[30.9716756438877...|489.09717664233835|
|[31.0662181616375...| 462.9655752595438|
|[31.3662121671876...|428.01688924626956|
|[31.5171218025062...| 281.5329792372231|
|[31.5316044825729...| 432.2689431608435|
|[31.5741380228732...| 558.6798148097353|
|[31.6005122003032...| 461.7333642290648|
|[31.6610498227460...|418.60320970932094|
|[31.7216523605090...| 350.9278121042478|
|[31.7242025238451...|509.84814507004216|
|[31.7656188210424...|500.94966223523716|
|[31.8093003166791...| 548.7629444748891|
|[31.8164283341993...| 518.3102630269218|
|[31.8209982016720...| 418.3981765140611|
|[31.8648325480987...|450.37032723765924|
|[31.9048571310136...| 491.5051533383414|
+--------------------+------------